Previous: [Overview and Selected Literature](01.ipynb) | [Table of Contents](00.ipynb) | Next: [The Linear Programming Problem](03.ipynb)

# The Willow Tree Lattice Handbook

## Mathematical Development of the Tree
The willow tree lattice is a computer library explicitly designed to mimic the behaviour of the __standard Brownian motion__. The algorithm does so in two steps. 

- First, it adopts a grid structure which extends in both time and space and which defines the nodes of the process. When probability is attached to them, these nodes represent, for each time step, a discrete approximation of the standard normal distribution.


- Then, it links sequences in adjacent time periods with a discrete Markov process.

The overall system converges to the standard Brownian motion as the grid becomes finer in both dimensions. Within the willow tree framework, space and time steps do not have any reciprocal dependence. On the one hand, the steps, particularly in the time dimension, need not be uniform; on the other, to achieve convergence the double limit is required.

## Curran's Methodology

To setup the tree, Curran [4] introduces the following methodology.

- First, build a __bidimensional grid__, whose nodes in time and space represent the possible states the paths of the process can transverse, as a function of $\mathbf{z}$ and $\mathbf{t}$. $\mathbf{z}$ is an $m$-valued vector of variates representative of the standard normal distribution, obtained by dividing the stratum of the distribution into $m$ intervals and letting a single value speak for the corresponding stratum. $\mathbf{t}$ is an $(n+1)$-valued vector of discrete time variables freely selected by the user.


- Then, determine the __likelihood to reach the states__. This step requires an $m$-valued vector $\mathbf{q}$ and a sequence of $m\times m$ matrices $\mathbf{P}$ representing the probabilities to transit from the previous node(s) to the next ones. 

The result is a process, $Y$, which converges to the standard Brownian motion in the limit.

### Step 1: Setting Up the Grid

The grid is an $m\times (n+1)$ object, function of vectors $\mathbf{z}$ and $\mathbf{t}$.

#### Vector $\mathbf{z}$

$\mathbf{z}$, the vector of representative standard normal variates, has values selected through _stratified sampling_ [6]. This technique, which Curran had already used in a previous paper [2], refers to the process of sampling from a population previously divided into mutually exclusive and collectively exhaustive subgroups, or _strata_. Let $\Phi$ denote the standard normal cumulative distribution function on the real line, and $\Phi^{-1}$ its inverse. Given probabilities $\{q_i\}_{i=1}^{m}$ summing to 1, define a sequence $\{a_i\}_{i=0}^{m}$ of quantiles of the standard normal distribution as follows:

$$
\begin{align*}
a_0&=-\infty=\Phi^{-1}(0),\\
a_1&=\Phi^{-1}(q_1),\\
a_2&=\Phi^{-1}(q_1+q_2),\\
&\dots\\
a_m&=\Phi^{-1}\big(\sum_{i=1}^{m} q_i\big)=\Phi^{-1}(1),
\end{align*}
$$

with $a_m=+\infty$. The strata are then the intervals:

$$
A_1 = (a_0,a_1],\quad A_2 = (a_1,a_2],\quad \dots,\quad A_m = (a_{m-1},a_m),
$$

with such quantiles as extremes. By construction, the probability of selecting a value belonging to stratum $A_i$ is $\Phi(a_i)-\Phi(a_{i-1})=q_i$.
Inside each interval, a single variable $z$ is picked as representative. Curran determines the sequence $\{z_i\}_{i=1}^m$ according to

$$
z_i = \Phi^{-1}\bigg(\frac{i-0.5}{m}\bigg),
$$

with equal probability $q_i = \mathbb{P}(Z = z_i)$ attached to each variable

$$
q_i = \frac{1}{m}.
$$

These probabilities are collected in a vector $\mathbf{q}$. The obtained pairs $\big\{\big(z_i,q_i\big)\big\}_{i=1}^m$ are a discrete approximation of the standard normal density function.

> __Figure 2__ depicts the outlined sampling procedure, with $m=11$ and uniform probability. The dashed lines are the bounds $\{a_i\}_{i=0}^m$ of the intervals, the dots the representative variates.


#### Figure 2: Stratified Sampling*
![stratified-sampling](../img/figure-2.png)
_* Adapted from Ho [8]._

#### Vector $\mathbf{t}$

$\mathbf{t}$, the $(n+1)$-valued vector of discrete time variables, has components $\{t_k\}_{k=0}^n$, with $0 = t_0 < t_1 < \dots < t_n = T$, representing a partition of the time interval $[0,T]$. $T$ is the ending date determined by the user.

The sequence of time variables must be strictly monotonic increasing. Hence, if one defines a $n$-valued vector $\mathbf{h}$ collecting the increments of adjacent time variables, $\{h_{k-1,k}\}_{k=1}^{n}$, with $h_{k-1,k}=t_k-t_{k-1}$, such vector must be strictly positive, or $\mathbf{h} \gg \mathbf{0}$. Moreover, a date is given by the sum of the increments up to that point:

$$
t_k = \sum_{j=1}^k h_{j-1,j},
$$

for $j=1,\dots,k$. The algorithm imposes no restriction on the size of the increments. Thus, time steps need not be uniform, a feature which grants flexibility in option pricing.

#### The Bidimensional Grid

With $\mathbf{z}$ and $\mathbf{t}$ determined, the $m\times(n+1)$ grid (a bidimensional matrix) is set up as follows. 

- For each $i=1,\dots,m$ and for each $k=0,\dots,n$ define, in accordance with the evolution of the standard Brownian motion, the element

$$
z_i\sqrt{t_k}.
$$

- When all the elements are collected into a matrix, the result is

$$
\mathbf{G}=
\begin{bmatrix}
0 & z_1\sqrt{t_1} & \dots & z_1\sqrt{t_n}\\
\vdots & \vdots & \ddots & \vdots\\
0 & z_m\sqrt{t_1} & \dots & z_m\sqrt{t_n}\\
\end{bmatrix}
$$

Each row of the grid can be seen as a slice in the time space, as it represents a snapshot of the state space at a particular date. Each column, instead, as a slice in the state space, as it represents the evolution of a single state through time. Obviously, each element in the first column is equal to zero, as $t_0=0$ and $z_i\sqrt{0}=0$. Hence, when plotted against time, the first column of the matrix will reduce to a single point, the origin.

### Step 2: Linking the States through Time

With the grid in place, the next step is to determine the sequences of transition probabilities linking states in adjacent time periods.

This step requires to define a _time-inhomogeneous (discrete-time) Markov chain_ $X_k$,  on the state space $\{1,2,\dots,m\}$. $\{X_k\}_{k=0}^{n}$ is a stochastic process with the Markov property, so the conditional probability distribution of the future nodes of the process only depends on the present state, while the past history of such process is irrelevant. It is a discrete-time chain, because the system only varies at discrete instances, and dependence only exists between adjacent time slices (as in a chain, in which only contiguous rings are connected). It is time-inhomogeneous, because the conditional probability distribution is not independent of the specific time period in which the prediction is made. Formally [10],

> __Definition.__ A _Markov chain_ is a discrete-time stochastic process $X_k$, $k \geq 0$, such that each random variable $X_k$ takes on value in a discrete set S (typically $S=\mathbb{N}$) and

>$$
\mathbb{P}(X_{k+1} = j \mid X_k = i,\ X_{k-1} = i_{k-1},\ \dots,\ X_0 = i_0) = \mathbb{P}(X_{k+1} = j \mid X_k = i),
$$

>for all $k \geq 0$, and for all $j$, $i$ and $\{i_k\}_{k=0}^{n-1} \in S$. Moreover, if $\mathbb{P}(X_{k+1} = j \mid X_k = i) = p_{ij}^k$ is dependent on $k$, the particular time period, then $X$ is time-inhomogeneous. The superscript $k$ in $p_{ij}^k$ marks this dependence.

#### The Transition Matrices

A time-inhomogeneous Markov chain is defined by a sequence of _transition matrices_ $\mathbf{P}^{(k)}=\big[p_{ij}^k\big]$, for $k = 0,\dots,n-1$. These square matrices, of dimensions $m\times m$, collect the probabilities to transit from a state $i$, in the previous time period, to a state $j$, in the next one, during a particular time interval $(t_k,t_{k+1})$. The superscript always denotes the starting date, so $k$ relates to the period ($t_k$, $t_{k+1}$).

Each transition matrix $\mathbf{P}^{(k)}$, for $k = 0,\dots,n-1$, has extended form:

$$
\mathbf{P}^{(k)}=
\begin{bmatrix}
p_{11}^k & p_{12}^k & \dots & p_{1m}^k\\
p_{21}^k & p_{22}^k & \dots & p_{2m}^k\\
\vdots & \vdots & \ddots & \vdots\\
p_{m1}^k & p_{m2}^k & \dots & p_{mm}^k\\
\end{bmatrix}
$$

In a transition matrix, subscript $i$ in each $p_{ij}^k$ denotes both the particular row of the matrix and the starting state, whereas subscript $j$ denotes both the column and the ending state. Each row of $\mathbf{P}^{(k)}$ collects the probabilities to transit from state $i$ to any state $j$, while each column the probabilities to end at state $j$ starting from any state $i$.

Transition matrices are _stochastic matrices_: all their entries are nonnegative, and each row (the marginal probabilities) sums to 1, or $\sum_{j=1}^m p_{ij}=1$, for all $i$. When also each column sums to 1, so that $\sum_{i=1}^m p_{ij}=1$, for all $j$, holds, the matrices are said to be _doubly stochastic_.

#### Vector $\mathbf{q}$

Matrix $\mathbf{P}^{(0)}$ represents a special case in the sequence. At the current date, $k=0$, there is no uncertainty about the starting node, $0$. Hence, the matrix $\big[p_{0j}^0\big]$ boils down to row vector $\mathbf{q}$, or $\{q_i\}_{i=1}^m$, collecting the probabilities to reach any state $j$ from current state $0$.

Vector $\mathbf{q}$ can be user-defined, as explained in the previous paragraph, or may be retrieved from empirical data, together with vector $\mathbf{z}$, through a calibration procedure (see [Xu, Hong, and Qin's Methodology](06.ipynb)).


The probability of reaching a particular state $j$ from current node $0$ may also be interpreted as that of picking a particular variable from a distribution through stratified sampling. Thus, $\mathbf{q}$ is also the vector collecting the discrete probabilities $q_i=\mathbb{P}(Z = z_i)$ of selecting the representative variates of a standard Brownian motion in the time interval ($0,t_1$).

Previous: [Overview and Selected Literature](01.ipynb) | [Table of Contents](00.ipynb) | Next: [The Linear Programming Problem](03.ipynb)